In [2]:
import gen3_data_validator
from gen3_data_validator.logging_config import setup_logging
setup_logging()

## Reading in xlsx data and writing to json
- xlsx data comes from xlsx manifest file created from acdc_submission_template

In [6]:
# ResolverClass = gen3_data_validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
xlsxData = gen3_data_validator.ParseXlsxMetadata(xlsx_path = "/Users/harrijh/projects/gen3-data-validator/data/lipid_metadata_example.xlsx", skip_rows=1)
xlsxData.parse_metadata_template()
xlsxData.write_dict_to_json(xlsx_data_dict=xlsxData.xlsx_data_dict, output_dir="/Users/harrijh/projects/gen3-data-validator/data/restricted/lipid_metadata_example")

## Testing Linkage

In [3]:
Data = gen3_data_validator.ParseData(data_folder_path = "../tests/data/fail")

In [ ]:
Data.file_path_list

In [5]:
# pass instance
Data = gen3_data_validator.ParseData(data_folder_path = "../tests/data/pass")
Resolver = gen3_data_validator.ResolveSchema(schema_path = "../tests/schema/gen3_test_schema.json")
Resolver.resolve_schema()
Linkage = gen3_data_validator.TestLinkage(schema_resolver = Resolver, data_parser = Data)

=== Validating Config Map ===
Root Node = ['subject']
Config Map Validated
=== Validating Links ===
Entity 'metabolomics_file' has 0 invalid foreign keys: []
Entity 'medical_history' has 0 invalid foreign keys: []
Entity 'metabolomics_assay' has 0 invalid foreign keys: []
Entity 'sample' has 0 invalid foreign keys: []
Entity 'subject' has 0 invalid foreign keys: []


In [6]:
# fail instance
Data = gen3_data_validator.ParseData(data_folder_path = "../tests/data/fail")
Resolver = gen3_data_validator.ResolveSchema(schema_path = "../tests/schema/gen3_test_schema.json")
Resolver.resolve_schema()
Linkage = gen3_data_validator.TestLinkage(schema_resolver = Resolver, data_parser = Data)

=== Validating Config Map ===
Root Node = ['subject']
Config Map Validated
=== Validating Links ===
Entity 'metabolomics_file' has 1099 invalid foreign keys: ['metabolomics_assay_356580ff6d', 'metabolomics_assay_44f829fa47', 'metabolomics_assay_974d137216', 'metabolomics_assay_3d1f400b27', 'metabolomics_assay_d1cd2f492c', 'metabolomics_assay_c025b20da0', 'metabolomics_assay_439725d38f', 'metabolomics_assay_d0350804b1', 'metabolomics_assay_63cef60fa4', 'metabolomics_assay_78465fe5b1', 'metabolomics_assay_3754fe418d', 'metabolomics_assay_0cdd244c6e', 'metabolomics_assay_40a94ece37', 'metabolomics_assay_adc5f88af9', 'metabolomics_assay_b646004109', 'metabolomics_assay_69bcc995f0', 'metabolomics_assay_37be2a2136', 'metabolomics_assay_119df8af52', 'metabolomics_assay_8dedaeccc1', 'metabolomics_assay_b353a7f9b8', 'metabolomics_assay_ebe904af55', 'metabolomics_assay_5bed5ab90c', 'metabolomics_assay_1417aac36c', 'metabolomics_assay_4e133f8d44', 'metabolomics_assay_38d2765ae0', 'metabolomics_as

In [9]:
Resolver = gen3_data_validator.ResolveSchema(schema_path = "../tests/schema/gen3_test_schema.json")
Resolver.resolve_schema()

In [10]:
Resolver.nodes

['demographic.yaml',
 'project.yaml',
 'serum_marker_assay.yaml',
 'alignment_workflow.yaml',
 'imaging_file.yaml',
 'lipidomics_assay.yaml',
 'metabolomics_file.yaml',
 'acknowledgement.yaml',
 'medical_history.yaml',
 '_definitions.yaml',
 '_settings.yaml',
 'blood_pressure_test.yaml',
 'genomics_assay.yaml',
 'variant_file.yaml',
 'program.yaml',
 'serum_marker_file.yaml',
 'proteomics_assay.yaml',
 'sample.yaml',
 'unaligned_reads_file.yaml',
 '_terms.yaml',
 'aligned_reads_index_file.yaml',
 'variant_workflow.yaml',
 'proteomics_file.yaml',
 'exposure.yaml',
 'metabolomics_assay.yaml',
 'lipidomics_mapping_file.yaml',
 'lipidomics_file.yaml',
 'aligned_reads_file.yaml',
 'lab_result.yaml',
 'medication.yaml',
 'publication.yaml',
 'subject.yaml',
 'core_metadata_collection.yaml']

You can also bypass the data attribute in the linkage class and input your own data_map and config_map

In [7]:
# # Using the Linkage class which has the resolved schema with custom data and config
# config_map = {
#     "samples": {"primary_key": "sample_id", "foreign_key": "subject_id"},
#     "files": {"primary_key": "file_id", "foreign_key": "sample_id"},
#     "subjects": {"primary_key": "subject_id", "foreign_key": "project_id"},
#     "project": {"primary_key": "project_id", "foreign_key": None}
# }

# data_map = {
#     "samples": [
#         {"sample_id": "sample_1", "subject_id": "subject_9"},
#         {"sample_id": "sample_2", "subject_id": "subject_3"},  # Invalid FK
#         {"sample_id": "sample_3", "subject_id": "subject_4"}, # Invalid FK
#         {"sample_id": "sample_4", "subject_id": "subject_5"} # Invalid FK
#     ],
#     "files": [
#         {"file_id": "file_1", "sample_id": "sample_1"},
#         {"file_id": "file_2", "sample_id": "sample_27"}  # Invalid FK
#     ],
#     "subjects": [
#         {"subject_id": "subject_1", "project_id": "project_1"},  
#         {"subject_id": "subject_2", "project_id": "project_2"}, # Missing project 2
#     ],
#     "project": [
#         {"project_id": "project_1"}
#     ]
# }

# Linkage.validate_links(data_map, config_map)

# validation prototype


## Creating the validation class

In [12]:
import gen3_data_validator

resolver = gen3_data_validator.ResolveSchema(schema_path = "../tests/schema/gen3_test_schema.json")
resolver.resolve_schema()
data = gen3_data_validator.ParseData(data_folder_path = "../tests/data/fail")
validator = gen3_data_validator.Validate(data_map=data.data_dict, resolved_schema=resolver.schema_resolved)

In [13]:
validator.validate_schema()

{'metabolomics_file': [{'index_0': [{'index': 0,
     'validation_result': 'FAIL',
     'invalid_key': 'data_format',
     'schema_path': 'properties.data_format.enum',
     'validator': 'enum',
     'validator_value': ['wiff'],
     'validation_error': "True is not one of ['wiff']"},
    {'index': 0,
     'validation_result': 'FAIL',
     'invalid_key': 'data_type',
     'schema_path': 'properties.data_type.enum',
     'validator': 'enum',
     'validator_value': ['MS', 'MS/MS'],
     'validation_error': "'1' is not one of ['MS', 'MS/MS']"}]},
  {'index_1': [{'index': 1,
     'validation_result': 'PASS',
     'invalid_key': None,
     'schema_path': None,
     'validator': None,
     'validator_value': None,
     'validation_error': None}]},
  {'index_2': [{'index': 2,
     'validation_result': 'PASS',
     'invalid_key': None,
     'schema_path': None,
     'validator': None,
     'validator_value': None,
     'validation_error': None}]},
  {'index_3': [{'index': 3,
     'validation_

In [14]:
validator.make_keymap()

{'metabolomics_file': ['index_0',
  'index_1',
  'index_2',
  'index_3',
  'index_4',
  'index_5',
  'index_6',
  'index_7',
  'index_8',
  'index_9',
  'index_10',
  'index_11',
  'index_12',
  'index_13',
  'index_14',
  'index_15',
  'index_16',
  'index_17',
  'index_18',
  'index_19',
  'index_20',
  'index_21',
  'index_22',
  'index_23',
  'index_24',
  'index_25',
  'index_26',
  'index_27',
  'index_28',
  'index_29',
  'index_30',
  'index_31',
  'index_32',
  'index_33',
  'index_34',
  'index_35',
  'index_36',
  'index_37',
  'index_38',
  'index_39',
  'index_40',
  'index_41',
  'index_42',
  'index_43',
  'index_44',
  'index_45',
  'index_46',
  'index_47',
  'index_48',
  'index_49',
  'index_50',
  'index_51',
  'index_52',
  'index_53',
  'index_54',
  'index_55',
  'index_56',
  'index_57',
  'index_58',
  'index_59',
  'index_60',
  'index_61',
  'index_62',
  'index_63',
  'index_64',
  'index_65',
  'index_66',
  'index_67',
  'index_68',
  'index_69',
  'index_

In [15]:
data.data_dict

{'metabolomics_file': [{'alternate_timepoint': '1a914a1577',
   'baseline_timepoint': True,
   'cv': 56.94475432813319,
   'data_category': 'mass spec analysed',
   'data_format': True,
   'data_type': '1',
   'file_format': 'e387cadce7',
   'file_name': 'dummy_metab',
   'file_size': 87,
   'ga4gh_drs_uri': '150bf4b457',
   'md5sum': '756c381b71c2a7d346c72998ab334c00',
   'metabolomic_unit': 'pmol/mL',
   'metabolomics_assays': {'submitter_id': 'metabolomics_assay_356580ff6d'},
   'submitter_id': 'metabolomics_file_547f3d4417',
   'type': 'metabolomics_file',
   'metabolomics_files': 'metabolomics_file_547f3d4417'},
  {'alternate_timepoint': '578a14ee53',
   'baseline_timepoint': True,
   'cv': 43.00152620641602,
   'data_category': 'mass spec analysed',
   'data_format': 'wiff',
   'data_type': 'MS/MS',
   'file_format': '47a60862ef',
   'file_name': 'dummy_metab',
   'file_size': 0,
   'ga4gh_drs_uri': '2beb8c16ea',
   'md5sum': '43640335849622369f4843b817c1da2e',
   'metabolomic_un

### Getting nested validation results
- returns a nested dictionary by entity/data node then by the row/index number, and then the validation objects

In [16]:
validation_dict = validator.validation_result
validation_dict

{'metabolomics_file': [{'index_0': [{'index': 0,
     'validation_result': 'FAIL',
     'invalid_key': 'data_format',
     'schema_path': 'properties.data_format.enum',
     'validator': 'enum',
     'validator_value': ['wiff'],
     'validation_error': "True is not one of ['wiff']"},
    {'index': 0,
     'validation_result': 'FAIL',
     'invalid_key': 'data_type',
     'schema_path': 'properties.data_type.enum',
     'validator': 'enum',
     'validator_value': ['MS', 'MS/MS'],
     'validation_error': "'1' is not one of ['MS', 'MS/MS']"}]},
  {'index_1': [{'index': 1,
     'validation_result': 'PASS',
     'invalid_key': None,
     'schema_path': None,
     'validator': None,
     'validator_value': None,
     'validation_error': None}]},
  {'index_2': [{'index': 2,
     'validation_result': 'PASS',
     'invalid_key': None,
     'schema_path': None,
     'validator': None,
     'validator_value': None,
     'validation_error': None}]},
  {'index_3': [{'index': 3,
     'validation_

In [17]:
validator.list_entities()

['metabolomics_file',
 'medical_history',
 'metabolomics_assay',
 'sample',
 'subject']

In [19]:
validator.list_index_by_entity("sample")

['index_0', 'index_1', 'index_2']

You can pull out a validation results for a specific entity with

In [20]:
validator.pull_entity("sample")

[{'index_0': [{'index': 0,
    'validation_result': 'FAIL',
    'invalid_key': 'freeze_thaw_cycles',
    'schema_path': 'properties.freeze_thaw_cycles.type',
    'validator': 'type',
    'validator_value': 'integer',
    'validation_error': "'10' is not of type 'integer'"},
   {'index': 0,
    'validation_result': 'FAIL',
    'invalid_key': 'sample_provider',
    'schema_path': 'properties.sample_provider.enum',
    'validator': 'enum',
    'validator_value': ['Baker', 'USYD', 'UMELB', 'UQ'],
    'validation_error': "45 is not one of ['Baker', 'USYD', 'UMELB', 'UQ']"},
   {'index': 0,
    'validation_result': 'FAIL',
    'invalid_key': 'sample_storage_method',
    'schema_path': 'properties.sample_storage_method.enum',
    'validator': 'enum',
    'validator_value': ['not stored',
     'ambient temperature',
     'cut slide',
     'fresh',
     'frozen, -70C freezer',
     'frozen, -150C freezer',
     'frozen, liquid nitrogen',
     'frozen, vapor phase',
     'paraffin block',
     '

You can pull validation results for a specific entity and then a specific index / row

In [21]:
validator.pull_index_of_entity("sample", "index_1")

[{'index': 1,
  'validation_result': 'FAIL',
  'invalid_key': 'freeze_thaw_cycles',
  'schema_path': 'properties.freeze_thaw_cycles.type',
  'validator': 'type',
  'validator_value': 'integer',
  'validation_error': "'76' is not of type 'integer'"},
 {'index': 1,
  'validation_result': 'FAIL',
  'invalid_key': 'sample_storage_method',
  'schema_path': 'properties.sample_storage_method.enum',
  'validator': 'enum',
  'validator_value': ['not stored',
   'ambient temperature',
   'cut slide',
   'fresh',
   'frozen, -70C freezer',
   'frozen, -150C freezer',
   'frozen, liquid nitrogen',
   'frozen, vapor phase',
   'paraffin block',
   'RNAlater, frozen',
   'TRIzol, frozen'],
  'validation_error': "'In the Pantry' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"}]

# Getting validation stats

In [22]:
validate_stats = gen3_data_validator.ValidateStats(validator)
stats_df = validate_stats.summary_stats()
stats_df

Total validation errors: 7


,entity,number_of_rows_with_errors,number_of_errors_per_entity
0,metabolomics_file,1,1
1,medical_history,2,2
2,metabolomics_assay,2,2
3,sample,2,2
4,subject,0,0


# Creating validation summary data

In [23]:
Summary = gen3_data_validator.ValidateSummary(validator) 
flattened_results_dict = Summary.flatten_validation_results()
flattened_results_dict

No validation results to flatten


[]

### Converting flattened dict to pandas

In [24]:
flatten_summary_pd = Summary.flattened_results_to_pd()
flatten_summary_pd

No validation errors found


""


### Collapsing flattened dict to pandas
- This collapsed data frame summarises common validation errors

In [25]:
collapse_df = Summary.collapse_flatten_results_to_pd()
collapse_df

No validation errors found


""


# Writing validation results to folder

In [ ]:
import os
output_dir = "../data/restricted/lipid_metadata/validation/"
os.makedirs(output_dir, exist_ok=True)


def write_dict_to_json(input_dict, output_dir, filename:str):
    with open(f"{output_dir}/{filename}.json", "w") as f:
        json.dump(input_dict, f)
    print(f"JSON files written to {output_dir}")

write_dict_to_json(validation_dict, output_dir, "validation_dict")
write_dict_to_json(flattened_results_dict, output_dir, "flattened_results_dict")

# Writing pandas df
stats_df.to_csv(f"{output_dir}/stats_df.csv")
flatten_summary_pd.to_csv(f"{output_dir}/flatten_summary_pd.csv")
collapse_df.to_csv(f"{output_dir}/collapse_df.csv")


In [ ]:
# Use this for writing tests

sample_validation_results = {
    'sample': [
        [
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'10' is not of type 'integer'"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_provider',
                'schema_path': 'properties.sample_provider.enum',
                'validator': 'enum',
                'validator_value': ['Baker', 'USYD', 'UMELB', 'UQ'],
                'validation_error': "45 is not one of ['Baker', 'USYD', 'UMELB', 'UQ']"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'Autoclave' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'76' is not of type 'integer'"
            },
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'In the Pantry' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 2,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ],
        [
            {
                'index': 3,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ]
    ]
}


